# Next Word Prediction:

### Importing Libraries:

In [72]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
import re

In [73]:
file = open(r"C:\Users\Vishesh Bansal\Desktop\New folder\text.txt")
lines = []

for i in file:
    lines.append(i)
    
print("The First five Line: ",'\n', lines[0],lines[1],lines[2],lines[3],lines[4])
print("The Last five Line:  ",'\n', lines[-5],lines[-4],lines[-3],lines[-2],lines[-1])

The First five Line:  
 The Project Gutenberg EBook of Metamorphosis, by Franz Kafka
 Translated by David Wyllie.
 
 This eBook is for the use of anyone anywhere at no cost and with
 almost no restrictions whatsoever.  You may copy it, give it away or

The Last five Line:   
 
 This Web site includes information about Project Gutenberg-tm,
 including how to make donations to the Project Gutenberg Literary
 Archive Foundation, how to help produce our new eBooks, and how to
 subscribe to our email newsletter to hear about new eBooks.


### Cleaning the data:

In [74]:
def cleantext():
    data = ""
    for data in lines:
        data = ' '.join(lines)
    data = data.replace('\n', '').replace('\r', '')
    data = re.sub(' +', ' ', data)
    data = re.sub(r"[^a-zA-Z0-9 ]", "", data)
    return data

data = cleantext()

### Tokenization:

In [75]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

In [76]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

3082


In [77]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)

The Length of sequences are:  25089


In [78]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [79]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [  1  44 113 282   5]
The responses are:  [ 44 113 282   5 503]


In [80]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Creating the Model:

In [81]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [82]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1, 10)             30820     
                                                                 
 lstm_8 (LSTM)               (None, 1, 1000)           4044000   
                                                                 
 lstm_9 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_8 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_9 (Dense)             (None, 3082)              3085082   
                                                                 
Total params: 16,164,902
Trainable params: 16,164,902
Non-trainable params: 0
_________________________________________________________________


### Compile The Model:

In [83]:
model.compile(loss="categorical_crossentropy", optimizer= tf.keras.optimizers.Adam(learning_rate=0.01))

### Callbacks:

In [84]:
checkpoint = ModelCheckpoint("nextword.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)


### Fit The Model:

In [85]:
model.fit(X, y, epochs=1, batch_size=64, callbacks=[checkpoint, reduce])

393/393 [==============================] - ETA: 0s - loss: 6.2348
Epoch 1: loss improved from inf to 6.23482, saving model to nextword1.h5
393/393 [==============================] - 64s 148ms/step - loss: 6.2348 - lr: 0.0100
